In [1]:
!pip install spacy

     |████████████████████████████████| 12.0MB 1.9MB/s eta 0:00:01


In [2]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 7.2MB/s eta 0:00:012    |███▊                            | 95.7MB 9.9MB/s eta 0:01:15     |███████████████████▍            | 501.1MB 8.6MB/s eta 0:00:38     |███████████████████████████▊    | 717.0MB 5.1MB/s eta 0:00:22


In [1]:
import spacy
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
tqdm.pandas()

/Users/mwasserman/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
from spacy.lang.en import English

In [4]:
df = pd.read_csv("yelp.csv")

In [5]:
df.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [6]:
nlp = spacy.load("en_trf_bertbaseuncased_lg")

In [7]:
#filtering stop words
from spacy.lang.en.stop_words import STOP_WORDS

def filter_stop_words(document, stop_words):
    '''takes in a document of tokenized text and returns
    a filtered document without stopwords'''
    return [word for word in document if str(word) not in stop_words]

In [8]:
df['tokens'] = df.text.progress_apply(lambda x: x.split())

100%|██████████| 10000/10000 [00:00<00:00, 80343.88it/s]


In [29]:
df['filtered_text'] = df.tokens.progress_apply(lambda x: filter_stop_words(x, STOP_WORDS))

100%|██████████| 10000/10000 [00:00<00:00, 41092.63it/s]


In [9]:
df['spacy'] = df.tokens.progress_apply(lambda x: nlp(' '.join(x)))

100%|██████████| 10000/10000 [1:12:10<00:00,  2.31it/s]


In [10]:
doc = df.spacy[0]
doc

My wife took me here on my birthday for breakfast and it was excellent. The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure. Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning. It looked like the place fills up pretty quickly so the earlier you get here the better. Do yourself a favor and get their Bloody Mary. It was phenomenal and simply the best I've ever had. I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it. It was amazing. While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious. It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete. It was the best "toast" I've ever had. Anyway, I can't wait to go back!

In [32]:
#acts as a list (can itterate range etc)
doc[:50]

My wife took birthday breakfast excellent. The weather perfect sitting outside overlooking grounds absolute pleasure. Our waitress excellent food arrived quickly semi-busy Saturday morning. It looked like place fills pretty quickly earlier better. Do favor Bloody Mary. It phenomenal simply best I've

In [11]:
#lemmatize (word or range)
doc[:50].lemma_

'My wife took me here on my birthday for breakfast and it was excellent . The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure . Our waitress was excellent and our food arrived quickly on the semi - busy Saturday morning . It looked like'

In [37]:
#document(or word similarity) cosine
doc.similarity(df.spacy[1])

0.9707812608337262

In [12]:
#document(or word similarity) cosine
doc.similarity(df.spacy[1])

0.9459664794040578

In [13]:
doc

My wife took me here on my birthday for breakfast and it was excellent. The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure. Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning. It looked like the place fills up pretty quickly so the earlier you get here the better. Do yourself a favor and get their Bloody Mary. It was phenomenal and simply the best I've ever had. I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it. It was amazing. While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious. It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete. It was the best "toast" I've ever had. Anyway, I can't wait to go back!

In [15]:
doc.vector + df.spacy[1].vector

array([-2.46215858e+01, -1.90274124e+01,  7.19377747e+01,  8.20927277e+01,
       -6.07937546e+01, -2.86508617e+01,  5.19720840e+01,  3.39311584e+02,
        1.35662460e+02, -1.39250595e+02,  9.76114426e+01, -2.19399475e+02,
       -2.27477913e+01,  2.81905792e+02, -7.11279144e+01,  1.42088409e+02,
        1.24929497e+02,  1.42418289e+00,  6.53858232e+00,  1.13176949e+02,
        2.10075607e+02, -3.05149651e+01,  1.66257286e+00,  3.60051025e+02,
        1.58933289e+02,  1.38643036e+01, -5.43144455e+01, -2.45997372e+01,
       -1.18162918e+00, -2.26899902e+02,  1.69192566e+02,  2.16278362e+01,
       -9.80020618e+00, -6.02358780e+01,  4.68729095e+01, -7.90744171e+01,
       -6.97083969e+01, -8.64268036e+01, -1.67329590e+02,  6.93418732e+01,
       -2.77517029e+02, -2.42228027e+02,  1.85425812e+02,  3.08228302e+00,
       -6.95421934e+00, -2.21024689e+02,  3.24142761e+01, -1.01236298e+02,
        4.68254700e+01, -1.02515106e+02, -9.72179794e+01,  1.02005737e+02,
        2.42138958e+00, -